In [1]:
import os
import json
import re
import time
import requests, psycopg2
import traceback
from bs4 import BeautifulSoup

In [7]:
root_url = "https://momandbaby.net/blog/page/"
url_list = set()

f = open("momandbaby_url.txt", "w")

for i in range(1,139):

    data = requests.get(root_url + str(i))
    soup = BeautifulSoup(data.text)
    content_soup = soup.find("div", {"class": "p-blog-list"})
    
    time.sleep(0.05)

    for link in content_soup.find_all('a'):
        link_str = str(link.get('href'))
        if '#' not in link_str and '?' not in link_str:
            url_list.add(link_str)


f.write(str(url_list))
f.close()

len(url_list)

1372

In [8]:
url_list

{'https://momandbaby.net/pregnant-mother/health/do-not-speak-when-your-wife-is-pregnant/',
 'https://momandbaby.net/parenting/childhealth/vaccine26032561/',
 'https://momandbaby.net/aesops-fables/flea-and-cow/',
 'https://momandbaby.net/parenting/raising-children13/nursery/',
 'https://momandbaby.net/parenting/child-development-and-health01year/8food/',
 'https://momandbaby.net/parenting/raising-children35/own-mood/',
 'https://momandbaby.net/lifestyle/fathers-day-greetings/',
 'https://momandbaby.net/lifestyle/events-that-mothers-have-to-face/',
 'https://momandbaby.net/lifestyle/weight-loss/',
 'https://momandbaby.net/pregnant-mother/cervix-nourishment-menus/',
 'https://momandbaby.net/lifestyle/car28032561/',
 'https://momandbaby.net/pregnant-mother/breast-milk/breast-milk07032561/',
 'https://momandbaby.net/parenting/child-development-and-health01year/finger-food01052561/',
 'https://momandbaby.net/lifestyle/issue29042561/',
 'https://momandbaby.net/pregnant-mother/health/footmassa

In [20]:
conn = psycopg2.connect(
    host="34.124.182.95",
    database="article_data",
    user="ai_user",
    password="{{PASSWORD}}"
)

cur = conn.cursor()
cur.execute("SELECT url from article_raw_data")
result = cur.fetchall()

inserted_list = []
for r in result:
    inserted_list.append(r[0])


for url in sorted(list(url_list)):

    
    if url in inserted_list:
        continue
    if '#' in url or '?' in url:
        continue

    print(url)
    try:
        data = requests.get(url)
        soup = BeautifulSoup(data.text)
    except:
        print("Dead, sleep 5 sec")
        time.sleep(5)
        continue
        
    content_soup = soup.find_all("div", {"class": "p-entry__body"})

    time.sleep(0.2)
    
    title = soup.title.get_text()

    p_tags = []
    for content in content_soup:
        p_tags.extend(content.find_all('p'))

    paragraph = []
    for p_tag in p_tags:
        text = p_tag.get_text()
        text = text.strip()
        
        if text.startswith("ที่มา"):
            break
        if text != "":
            if "\n" in text:
                tmp = text.split("\n")
                for t in tmp:
                    if len(t) > 150:
                        paragraph.append(t)
            else:
                if len(text) > 150:
                    paragraph.append(text)


    try:
        cur.execute("BEGIN")
        cur.execute(
            "INSERT INTO article_raw_data (url, category, title, content, paragraphs, raw_html, raw_json, remark) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)",
            (url, "Mom & Baby", title, "", json.dumps(paragraph, ensure_ascii=False), str(soup), "{}", "crawl page 1-138")
        )
        cur.execute("COMMIT")
    except psycopg2.Error as e:
        conn.rollback()
        print("Error: {}".format(e))
        
    inserted_list.append(url)

https://momandbaby.net/aesops-fables/flea-and-cow/
https://momandbaby.net/aesops-fables/the-ants-and-the-grasshopper/
https://momandbaby.net/aesops-fables/the-boy-who-cried-wolf/
https://momandbaby.net/aesops-fables/the-flies/
https://momandbaby.net/aesops-fables/the-greedy-dog/
https://momandbaby.net/baby-food/5-color-vegetables/
https://momandbaby.net/baby-food/8food24052561/
https://momandbaby.net/baby-food/baby-food13/apple1314032561/
https://momandbaby.net/baby-food/baby-food13/food24052561/
https://momandbaby.net/baby-food/baby-food13/goatmilk06072561/
https://momandbaby.net/baby-food/baby-food13/menu10042561/
https://momandbaby.net/baby-food/baby-food13/menu1316032561/
https://momandbaby.net/baby-food/baby-food13/menu190402561/
https://momandbaby.net/baby-food/baby-food13/menu20042561/
https://momandbaby.net/baby-food/baby-food13/menu21032561/
https://momandbaby.net/baby-food/baby-food13/sleep04062561/
https://momandbaby.net/baby-food/baby-food13/sweets01052561/
https://momandba

https://momandbaby.net/lifestyle/child-name/nickname-phu/
https://momandbaby.net/lifestyle/child-name/nickname-r/
https://momandbaby.net/lifestyle/child-name/nickname-s/
https://momandbaby.net/lifestyle/child-name/nickname-sh/
https://momandbaby.net/lifestyle/child-name/nickname-ss/
https://momandbaby.net/lifestyle/child-name/nickname-t/
https://momandbaby.net/lifestyle/child-name/nickname-tah/
https://momandbaby.net/lifestyle/child-name/nickname-th/
https://momandbaby.net/lifestyle/child-name/nickname-thh/
https://momandbaby.net/lifestyle/child-name/nickname-tho/
https://momandbaby.net/lifestyle/child-name/nickname-tth/
https://momandbaby.net/lifestyle/child-name/nickname-u/
https://momandbaby.net/lifestyle/child-name/nickname-w/
https://momandbaby.net/lifestyle/child-name/nickname-y-2/
https://momandbaby.net/lifestyle/child-name/nickname-y/
https://momandbaby.net/lifestyle/child8year01082562/
https://momandbaby.net/lifestyle/childrens-wear08062561/
https://momandbaby.net/lifestyle/ch

https://momandbaby.net/lifestyle/travel-pattaya/
https://momandbaby.net/lifestyle/travel-songkran-near-bangkok/
https://momandbaby.net/lifestyle/travel-temple-nan-province/
https://momandbaby.net/lifestyle/travel-to-mon-jam/
https://momandbaby.net/lifestyle/travel22032561/
https://momandbaby.net/lifestyle/tree-protects-snake/
https://momandbaby.net/lifestyle/truth03042561/
https://momandbaby.net/lifestyle/vitaminc/
https://momandbaby.net/lifestyle/washing-uniform/
https://momandbaby.net/lifestyle/water-from-washing-rice/
https://momandbaby.net/lifestyle/weight-loss/
https://momandbaby.net/lifestyle/welfare/
https://momandbaby.net/lifestyle/what-husbands-can-help-their-wives/
https://momandbaby.net/lifestyle/what-you-think-vs-what-is-about-kid-food/
https://momandbaby.net/lifestyle/what-you-think-vs-what-is-when-you-have-kids/
https://momandbaby.net/lifestyle/whats-on-the-checklist-for-newborn-stuff/
https://momandbaby.net/lifestyle/white-clay/
https://momandbaby.net/lifestyle/white-fru

https://momandbaby.net/parenting/child-development-and-health13year/foreign-language/
https://momandbaby.net/parenting/child-development-and-health13year/free/
https://momandbaby.net/parenting/child-development-and-health13year/how-to-knock/
https://momandbaby.net/parenting/child-development-and-health13year/howtopm25/
https://momandbaby.net/parenting/child-development-and-health13year/humpback-management-posture/
https://momandbaby.net/parenting/child-development-and-health13year/hunchbacked-child/
https://momandbaby.net/parenting/child-development-and-health13year/hygiene/
https://momandbaby.net/parenting/child-development-and-health13year/little-sleeping/
https://momandbaby.net/parenting/child-development-and-health13year/nasal-wash21052561/
https://momandbaby.net/parenting/child-development-and-health13year/plays-suitable-for-a-1-year-old-child/
https://momandbaby.net/parenting/child-development-and-health13year/runny-nose/
https://momandbaby.net/parenting/child-development-and-hea

https://momandbaby.net/parenting/raising-children01/baby-wearing-a-mask/
https://momandbaby.net/parenting/raising-children01/baby11072561/
https://momandbaby.net/parenting/raising-children01/bathing-baby/
https://momandbaby.net/parenting/raising-children01/breastfeeding-posture/
https://momandbaby.net/parenting/raising-children01/carrying-a-child/
https://momandbaby.net/parenting/raising-children01/child-crying-colic/
https://momandbaby.net/parenting/raising-children01/child-raising11032561/
https://momandbaby.net/parenting/raising-children01/clever-ball/
https://momandbaby.net/parenting/raising-children01/colic25032561/
https://momandbaby.net/parenting/raising-children01/cry05092562/
https://momandbaby.net/parenting/raising-children01/cry26062561/
https://momandbaby.net/parenting/raising-children01/crying20052561/
https://momandbaby.net/parenting/raising-children01/dietary-supplement-for-children612/
https://momandbaby.net/parenting/raising-children01/eat09042561/
https://momandbaby.n

...

https://momandbaby.net/parenting/raising-children35/help17082561/
https://momandbaby.net/parenting/raising-children35/heredity/
https://momandbaby.net/parenting/raising-children35/hold07082561/
https://momandbaby.net/parenting/raising-children35/home-school/
https://momandbaby.net/parenting/raising-children35/home-school02042561/
https://momandbaby.net/parenting/raising-children35/home-school31032561/
https://momandbaby.net/parenting/raising-children35/home30072561/
https://momandbaby.net/parenting/raising-children35/homework16072561/
https://momandbaby.net/parenting/raising-children35/hot-temper/
https://momandbaby.net/parenting/raising-children35/how-to-clean-toys/
https://momandbaby.net/parenting/raising-children35/how-to-raise-children-reasonably/
https://momandbaby.net/parenting/raising-children35/how-to-teach-children-to-solve-problems-on-their-own/
https://momandbaby.net/parenting/raising-children35/how-to-teach-your-child-to-be-a-good-person-and-have-a-happy-life/
https://moman

https://momandbaby.net/parenting/raising-children6/school15042561/
https://momandbaby.net/parenting/raising-children6/sign26072561/
https://momandbaby.net/parenting/raising-children6/solve-the-problem-by-yourself/
https://momandbaby.net/parenting/raising-children6/talking-sarcastically/
https://momandbaby.net/parenting/raising-children6/teach-children-to-accept-differences-in-society/
https://momandbaby.net/parenting/raising-children6/teach28032561/
https://momandbaby.net/parenting/raising-children6/the-child-is-kind-or-cant-deny-it/
https://momandbaby.net/parenting/raising-children6/thieving/
https://momandbaby.net/parenting/reading-the-book/
https://momandbaby.net/parenting/reduce-fever-for-children/
https://momandbaby.net/parenting/signs-of-nutritional-deficiency/
https://momandbaby.net/parenting/socialization-kids/
https://momandbaby.net/parenting/squeeze-toothpaste/
https://momandbaby.net/parenting/stutter-in-baby/
https://momandbaby.net/parenting/stuttering-problems/
https://moma

https://momandbaby.net/pregnant-mother/breast-milk/cannot-get-out-of-the-breast-pump/
https://momandbaby.net/pregnant-mother/breast-milk/culled-breast/
https://momandbaby.net/pregnant-mother/breast-milk/diarrhea23082561/
https://momandbaby.net/pregnant-mother/breast-milk/food-mom01122562/
https://momandbaby.net/pregnant-mother/breast-milk/food-to-increase-milk/
https://momandbaby.net/pregnant-mother/breast-milk/how-to-calculate/
https://momandbaby.net/pregnant-mother/breast-milk/how-to-reheat-frozen-breast-milk/
https://momandbaby.net/pregnant-mother/breast-milk/how-to-stock-breast-milk/
https://momandbaby.net/pregnant-mother/breast-milk/how-to-wake-the-child-into-the-breast/
https://momandbaby.net/pregnant-mother/breast-milk/howto26052561/
https://momandbaby.net/pregnant-mother/breast-milk/milk02112561/
https://momandbaby.net/pregnant-mother/breast-milk/milk03072561/
https://momandbaby.net/pregnant-mother/breast-milk/milk0507/
https://momandbaby.net/pregnant-mother/breast-milk/milk060

https://momandbaby.net/pregnant-mother/health/lose-weight11042561/
https://momandbaby.net/pregnant-mother/health/lower-abdominal-pain-during-pregnancy/
https://momandbaby.net/pregnant-mother/health/massage24042561/
https://momandbaby.net/pregnant-mother/health/maternal-pregnancy30032561/
https://momandbaby.net/pregnant-mother/health/medical08112561/
https://momandbaby.net/pregnant-mother/health/medications-can-pregnant-eat/
https://momandbaby.net/pregnant-mother/health/medicine25032561/
https://momandbaby.net/pregnant-mother/health/mocktail/
https://momandbaby.net/pregnant-mother/health/mom19062561/
https://momandbaby.net/pregnant-mother/health/mom23072561/
https://momandbaby.net/pregnant-mother/health/mother-is-pregnant/
https://momandbaby.net/pregnant-mother/health/mothers-are-pregnant/
https://momandbaby.net/pregnant-mother/health/mouth24062561/
https://momandbaby.net/pregnant-mother/health/nausea-and-vomiting-of-pregnancy/
https://momandbaby.net/pregnant-mother/health/oral-health-c

https://momandbaby.net/review/5-%e0%b9%80%e0%b8%84%e0%b8%a3%e0%b8%b7%e0%b9%88%e0%b8%ad%e0%b8%87%e0%b8%9f%e0%b8%b1%e0%b8%87%e0%b9%80%e0%b8%aa%e0%b8%b5%e0%b8%a2%e0%b8%87%e0%b8%ab%e0%b8%b1%e0%b8%a7%e0%b9%83%e0%b8%88%e0%b8%97%e0%b8%b2/
https://momandbaby.net/review/anti-hairloss-shampoo/
https://momandbaby.net/review/anti-mosquito-path/
https://momandbaby.net/review/antigen-test-kit/
https://momandbaby.net/review/baby-bag/
https://momandbaby.net/review/baby-bed-rails/
https://momandbaby.net/review/baby-bed/
https://momandbaby.net/review/baby-blanket/
https://momandbaby.net/review/baby-blender/
https://momandbaby.net/review/baby-bottle-brush/
https://momandbaby.net/review/baby-bottle-cleaner/
https://momandbaby.net/review/baby-bottle-sterilizer/
https://momandbaby.net/review/baby-carriers/
https://momandbaby.net/review/baby-chair-foldable/
https://momandbaby.net/review/baby-creams/
https://momandbaby.net/review/baby-cutlery-set/
https://momandbaby.net/review/baby-laundry-detergent/
https://